#**LABORATORIO 2 - MLP PYTORCH**

Nombre: Quispe Sucullani Jose David

Enlace de GitHub:

##*LABORATORIO 1*

###Construir un MLP, con pytorch, considerando lo siguiente:

- El MLP, debe servir para realizar clasificaciones de al menos 5 clases o tipos de difentes ambitos.

- El MLP debe contener al menos 50 unidades  neuronales en una capa oculta.

- Se debe entrenar por almenos 1000 epochs y explicar que sucede en este proceso.

- Se debe implementar un mecanismo de checkpoints para el proceso de entrenamiento que permita respaldar los valores calculados para los parametros, cada 20 epochs.

- Aplicar las formas (frameworks) de exportación de modelos Torchscript y ONNX, a traves de ejemplos.

- Utilizar de manera obligatoria objetos dataset y dataloader en la implementacion.

- Se debe aplicar las tecnicas de normalizacion, optimizacion y buenas practicas para lograr el menor valor de costo y el mayor valor de precision.


Se debe subir todo a un repositorio el cual debes ser compartido, ademas de subir todos los cuadernillos o codigo fuente generados.

In [ ]:
# IMPORTAMOS LAS LIBRERIAS

import torch
import torchvision
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm  # Importar tqdm para la barra de progreso
from torch.cuda.amp import autocast, GradScaler  # Para mixed precision training

In [ ]:
import torch

# Verifica si CUDA está disponible
print(f"¿CUDA disponible?: {torch.cuda.is_available()}")

# Nombre de la GPU
if torch.cuda.is_available():
    print(f"Nombre de la GPU: {torch.cuda.get_device_name(0)}")
    print(f"Versión de CUDA: {torch.version.cuda}")
else:
    print("¡No se detectó CUDA! Revisa la instalación de drivers y PyTorch.")

: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#**1: Lectura del Dataset**

Transformaciones: Redimensionamos los imagenes a 128x128 píxeles para convierten en tensores. La normalización se realiza para que los valores de píxeles estén en un rango adecuado para el entrenamiento del modelo.

In [ ]:
# Definir las transformaciones para las imágenes
from torchvision import datasets, transforms
import torch.utils.data

# Definir las transformaciones para el conjunto de datos
transform = transforms.Compose([
    transforms.Resize((50, 50)),  # Redimensionar las imágenes a 50x50 píxeles
    transforms.ToTensor(),  # Convertir las imágenes a tensores
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalización de las imágenes
])

train_data = datasets.ImageFolder(
    root='/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio1MLP/train',
    transform=transform
)
test_data = datasets.ImageFolder(
    root='/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio1MLP/test',
    transform=transform
)

print(f"Número de imágenes de entrenamiento: {len(train_data)}")
print(f"Número de imágenes de prueba: {len(test_data)}")
print(f"Clases detectadas: {train_data.classes}")


In [ ]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import random

# Definir las transformaciones para el conjunto de datos
transform = transforms.Compose([
    transforms.Resize((50, 50)),  # Redimensionar las imágenes a 50x50 píxeles
    transforms.ToTensor(),  # Convertir las imágenes a tensores
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalización de las imágenes
])

# Función para limitar el número de archivos de cada subcarpeta
def limit_dataset_size(root_dir, max_per_class=6000):
    # Diccionario para almacenar rutas de las imágenes seleccionadas por clase
    limited_dataset = []

    # Iteramos por las subcarpetas en root_dir (cada subcarpeta es una clase)
    for class_name in os.listdir(root_dir):
        class_dir = os.path.join(root_dir, class_name)
        if os.path.isdir(class_dir):
            # Obtenemos todas las imágenes en la subcarpeta
            all_images = os.listdir(class_dir)
            # Filtramos para obtener solo imágenes, excluyendo otros tipos de archivos
            all_images = [os.path.join(class_name, img) for img in all_images if img.endswith(('.png', '.jpg', '.jpeg'))]
            # Seleccionamos aleatoriamente 6000 imágenes
            selected_images = random.sample(all_images, min(len(all_images), max_per_class))
            limited_dataset.extend(selected_images)

    return limited_dataset



In [ ]:
# Definir la ruta del dataset
root_dir = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio1MLP/train'

# Obtener una lista de las rutas de imágenes limitadas a 6000 por clase
limited_image_paths = limit_dataset_size(root_dir, max_per_class=6000)

# Clase personalizada para cargar solo las imágenes seleccionadas
class CustomImageFolder(datasets.ImageFolder):
    def __init__(self, root, selected_files, transform=None):
        super(CustomImageFolder, self).__init__(root, transform=transform)
        # Filtrar el dataset para cargar solo las imágenes seleccionadas
        self.samples = [(os.path.join(root, img), self.class_to_idx[img.split('/')[0]]) for img in selected_files]
        self.targets = [s[1] for s in self.samples]  # Extraer las etiquetas

# Crear el dataset personalizado usando solo las imágenes seleccionadas
train_data = CustomImageFolder(root=root_dir, selected_files=limited_image_paths, transform=transform)

# Verificar el número de imágenes en el nuevo conjunto de entrenamiento
print(f"Número total de imágenes en el nuevo conjunto de entrenamiento: {len(train_data)}")

# Verificar el número de imágenes por clase
from collections import Counter
print("Distribución de imágenes por clase:", Counter(train_data.targets))

In [ ]:
# 1. Ver las etiquetas del dataset (clases):
# Ver las clases del dataset (etiquetas)
print(f"Clases en el conjunto de entrenamiento: {train_data.classes}")
print(f"Cantidad de clases: {len(train_data.classes)}")

In [ ]:
import torch
from collections import Counter

# Verificar el número total de imágenes en el conjunto de entrenamiento
num_total_images = len(train_data)
print(f"Número total de imágenes en el conjunto de entrenamiento: {num_total_images}")

# Verificar el número de clases (etiquetas)
num_classes = len(train_data.classes)
print(f"Número de clases: {num_classes}")

# Obtener las etiquetas de todas las imágenes (train_data.targets tiene las etiquetas de cada imagen)
class_distribution = Counter(train_data.targets)

# Mostrar cuántas imágenes hay por clase
for class_idx, count in class_distribution.items():
    print(f"Clase '{train_data.classes[class_idx]}' tiene {count} imágenes")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# Función para mostrar imágenes
def imshow(img):
    img = img / 2 + 0.5  # Desnormalizar la imagen
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Mostrar algunas imágenes del conjunto de entrenamiento
dataiter = iter(train_data)
images, labels = next(dataiter)

# Mostrar las imágenes
imshow(torchvision.utils.make_grid(images))

# Mostrar las etiquetas correspondientes
# Check if labels is an integer and convert it to a list if necessary
if isinstance(labels, int):
    labels = [labels]

print('Etiquetas:', ' '.join(f'{train_data.classes[labels[j]]}' for j in range(len(labels))))

In [ ]:
# Preparar los DataLoaders en formato de diccionario
dataloader = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, pin_memory=True),
    'test': torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True)
}

# Mostrar la cantidad de imágenes de entrenamiento y prueba
print(f"Número de imágenes de entrenamiento: {len(train_data)}")
print(f"Número de imágenes de prueba: {len(test_data)}")

In [ ]:
# Obtener un lote de imágenes y etiquetas del DataLoader de entrenamiento
images, labels = next(iter(dataloader['train']))

# Mostrar el tamaño del lote de imágenes y etiquetas
print(f"Tamaño del lote de imágenes: {images.shape}")  # Dimensiones del tensor de imágenes
print(f"Tamaño del lote de etiquetas: {labels.shape}")  # Dimensiones del tensor de etiquetas

# Ver la primera etiqueta en el lote y su correspondencia con la clase
print(f"Primera etiqueta en el lote: {labels[0].item()}")  # Mostrar el valor de la primera etiqueta
print(f"Clase correspondiente: {train_data.classes[labels[0]]}")  # Mostrar la clase asociada

Aplicamos la utilización de los objetos de dataset y DataLoader para poder facilitar los dataset de grandes magnitudes dividiendo en lotes de batch_size

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

# Obtener las clases del dataset
classes = train_data.classes

# Convertir las imágenes y etiquetas a arrays numpy para facilitar el procesamiento
def imshow(img):
    img = img / 2 + 0.5     # Desnormalizar
    npimg = img.numpy()
    return np.transpose(npimg, (1, 2, 0))

# Mostrar 10 imágenes aleatorias
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    index = random.randint(0, len(train_data) - 1)
    img, label = train_data[index]
    plt.imshow(imshow(img))
    plt.title(classes[label])
    plt.axis('off')

plt.show()

#**2: Construcción del Modelo MLP**

Ahora realizaremos la arquitectura del MLP, asegurando que cumpla con las especificaciones: al menos 50 unidades neuronales en una capa oculta y una capa de salida con 5 unidades, una para cada clase de planta.

In [ ]:

import torch.nn as nn
import torch.nn.functional as F

# Definir un bloque secuencial de capas fully connected con activación y dropout
def fc_block(input_size, output_size, dropout_rate=0.5):
    return nn.Sequential(
        nn.Linear(input_size, output_size),  # Capa totalmente conectada (fully connected)
        nn.ReLU(),                           # Activación ReLU
        nn.Dropout(dropout_rate)             # Dropout para regularización
    )

class MLP(nn.Module):
    def __init__(self, input_size=50*50*3, n_classes=5):
        super(MLP, self).__init__()
        # Definir las capas fully connected utilizando bloques
        self.fc1 = fc_block(input_size, 50, dropout_rate=0.5)  # Capa de entrada con 7500 unidades y 50 neuronas
        self.fc2 = fc_block(50, 30, dropout_rate=0.5)         # Capa oculta con 30 unidades de neuronas
        self.fc3 = nn.Linear(30, n_classes)                    # Capa de salida con 5 clases (una por cada planta)

    def forward(self, x):
        x = x.view(-1, 50 * 50 * 3)  # Aplanar las imágenes
        x = self.fc1(x)              # Pasar a través del primer bloque fully connected
        x = self.fc2(x)              # Pasar a través del segundo bloque fully connected
        x = self.fc3(x)              # Pasar a través de la capa de salida
        return x


In [ ]:
# Configuración del dispositivo (GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#**3: Configuración del Entrenamiento**

Entrenaremos el modelo durante 100 epochs, implementando un mecanismo de checkpoints para guardar el modelo cada 20 epochs. Además, aplicaremos técnicas de optimización y buenas prácticas.

In [ ]:
!pip install tqdm

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

# Ruta donde se guardarán los checkpoints
checkpoint_dir = "/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio1MLP"

# Función para guardar un checkpoint en la ruta especificada
def save_checkpoint(epoch, model, optimizer, loss, filename='checkpoint.pth'):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    full_path = checkpoint_dir + filename  # Construir la ruta completa
    torch.save(checkpoint, full_path)
    print(f'Checkpoint guardado en: {full_path}')



# Función para cargar un checkpoint desde la ruta especificada
def load_checkpoint(filename, model, optimizer):
    full_path = checkpoint_dir + filename  # Construir la ruta completa
    checkpoint = torch.load(full_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f'Checkpoint cargado desde epoch {epoch} con pérdida {loss:.4f}')
    return epoch, loss

Entrenamiento desde cero y guardado de checkpoints:

In [ ]:
# Función para entrenar el modelo y guardar checkpoints automáticamente
def fit_train(model, dataloader, epochs=10, lr=1e-3, checkpoint_interval=5):
    model.to(device)  # Mover el modelo al dispositivo (CPU o GPU)

    # Optimizador Adam y función de pérdida CrossEntropy
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(1, epochs + 1):
        # Inicializar listas para pérdidas y precisiones del entrenamiento
        model.train()
        train_loss, train_acc = [], []

        # Barras de progreso para los datos de entrenamiento
        bar = tqdm(dataloader['train'], desc=f"Epoch {epoch}/{epochs} - Training")

        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)  # Mover los datos al dispositivo

            # Reiniciar gradientes
            optimizer.zero_grad()

            # Forward pass (propagación hacia adelante)
            y_hat = model(X)

            # Cálculo de la pérdida
            loss = criterion(y_hat, y)

            # Backward pass (retropropagación) y actualización del optimizador
            loss.backward()
            optimizer.step()

            # Guardar el valor de la pérdida
            train_loss.append(loss.item())

            # Cálculo de precisión
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)

            # Actualización de la barra de progreso
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")

        # Validación del modelo después de cada época
        model.eval()  # Cambiar a modo de evaluación
        val_loss, val_acc = [], []

        with torch.no_grad():  # Desactivar el cálculo del gradiente
            bar = tqdm(dataloader['test'], desc=f"Epoch {epoch}/{epochs} - Validation")

            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)

                # Forward pass en modo evaluación
                y_hat = model(X)

                # Cálculo de la pérdida de validación
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())

                # Cálculo de la precisión de validación
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)

                # Actualización de la barra de progreso
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")

        # Mostrar métricas al final de la época
        print(f"Epoch {epoch}/{epochs} - Train loss: {np.mean(train_loss):.5f} - Validation loss: {np.mean(val_loss):.5f} - Train acc: {np.mean(train_acc):.5f} - Validation acc: {np.mean(val_acc):.5f}")

        # Guardar checkpoint cada cierto número de epochs
        if epoch % checkpoint_interval == 0:
            save_checkpoint(epoch, model, optimizer, np.mean(train_loss), filename=f'MLP_checkpoint_epoch_{epoch}.pth')


In [ ]:
# Crear instancia del modelo MLP y entrenar desde cero
model = MLP()
fit_train(model, dataloader, epochs=10, lr=1e-3, checkpoint_interval=2)

Cargar un modelo desde un checkpoint y continuar el entrenamiento:

In [ ]:
# Función para continuar el entrenamiento desde un checkpoint
def fit_train_continue(model, dataloader, checkpoint_filename, epochs=10, lr=1e-3, checkpoint_interval=5):
    model.to(device)  # Mover el modelo al dispositivo (CPU o GPU)

    # Optimizador Adam y función de pérdida CrossEntropy
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    # Cargar el checkpoint
    start_epoch, _ = load_checkpoint(checkpoint_filename, model, optimizer)

    for epoch in range(start_epoch + 1, epochs + 1):
        # Inicializar listas para pérdidas y precisiones del entrenamiento
        model.train()
        train_loss, train_acc = [], []

        # Barras de progreso para los datos de entrenamiento
        bar = tqdm(dataloader['train'], desc=f"Epoch {epoch}/{epochs} - Training")

        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)  # Mover los datos al dispositivo

            # Reiniciar gradientes
            optimizer.zero_grad()

            # Forward pass (propagación hacia adelante)
            y_hat = model(X)

            # Cálculo de la pérdida
            loss = criterion(y_hat, y)

            # Backward pass (retropropagación) y actualización del optimizador
            loss.backward()
            optimizer.step()

            # Guardar el valor de la pérdida
            train_loss.append(loss.item())

            # Cálculo de precisión
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)

            # Actualización de la barra de progreso
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")

        # Guardar checkpoint cada cierto número de epochs
        if epoch % checkpoint_interval == 0:
            save_checkpoint(epoch, model, optimizer, np.mean(train_loss), filename=f'MLP_checkpoint_epoch_{epoch}.pth')


##Evalucion del Modelo MLP

In [ ]:
def evaluate(model, dataloader):
    model.eval()  # Establecer el modelo en modo de evaluación
    model.to(device)  # Mover el modelo al dispositivo adecuado (CPU/GPU)

    bar = tqdm(dataloader['test'], desc="Evaluating")  # Barra de progreso para la evaluación
    accuracy = []  # Lista para almacenar las precisiones

    with torch.no_grad():  # Desactivar el cálculo de gradientes
        for batch in bar:
            X, y = batch  # Obtener las entradas (X) y etiquetas (y) del lote
            X, y = X.to(device), y.to(device)  # Mover los datos al dispositivo

            # Realizar una predicción con el modelo
            y_hat = model(X)

            # Calcular la precisión
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            accuracy.append(acc)

            # Actualizar la barra de progreso con la precisión promedio actual
            bar.set_description(f"acc {np.mean(accuracy):.5f}")

    # Imprimir la precisión final de la evaluación
    print(f"Evaluación completada - Precisión promedio: {np.mean(accuracy):.5f}")


In [ ]:
# Evaluar el modelo MLP
evaluate(model, dataloader)

In [ ]:
print(model)

#CONCLUSIÓN Y RESULTADOS

- **HiperParámetros**: Los hiperparámetros que se utilizaron fueron el valor de alpha coeficiente de aprendizaje de 0.001.

Para el modelo del MLP utilizamos 128*128*3 de entrada, un 50 capas ocultas y la última capa de salida que es de 5 clases diferentes.


Sobre los resultamos aún no se especificó los resultados correctos ya que se hacen muchas pruebas con diferentes tipos de hiperparámetros por lo que tendremos que entrenar muchas veces para sacar el mejor resultado.

####Tecnicas y Prácticas

- Regularización: Se añadió Dropout en el modelo para reducir el sobreajuste al apagar aleatoriamente un porcentaje de neuronas durante el entrenamiento.

- Optimización: Para la optimización del modelo aplicamos un optimizador que es el Adan que adapta las tasas de aprendizaje y estabiliza el entrenamiento. Existen otros que podriamos utilizar como Learning Rate Scheduling o Gradient Clipping, pero la más famosa que se utiliza es el Adan por lo cual percatamos a utilizar para la optimización del entrenamiento.

- Buenas Prácticas:

  . Utilizamos DataLoader para manejar el batching y aleatorización de los datos de entrenamiento y prueba
  
  . Checkpoints: Guardamos el estado del modelo y el optimizador cada 20 epochs para recuperar el progreso en caso de interrupciones.

  . Exportación del Modelo: Exportamos el modelo en dos
  formatos: TorchScript para su uso en producción y ONNX para interoperabilidad con otras plataformas.


####Entrenamiento y Evaluación

- Entrenamiento: El modelo se entrena durante 1000 epochs, donde en cada epoch se realiza el forward pass, cálculo de pérdida, retropropagación y actualización de pesos. La pérdida se imprime al final de cada epoch.

- Evaluación: Después del entrenamiento de cada epoch, se evalúa el modelo en el conjunto de prueba para calcular la precisión y verificar el rendimiento.